<a href="https://colab.research.google.com/github/custom-hyper/defi_lama/blob/master/DeFi_Lama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [ ]:
!pip install requests
!pip install json


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


## Token  Coverage Dataset

In [40]:
#@title Extract Current Data – All Protocols { display-mode: "code" }
import requests
import json
from google.colab import data_table
import pandas as pd
import logging
import pandas as pd
import logging
import time




#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
table = json.loads(output)
df = pd.json_normalize(table)
slug = df['slug']

path = "https://api.llama.fi/protocol/"
frames = []


try:
  for i in slug[:3]:
    response = requests.get(path + i)
    output = response.text
    DeFi = json.loads(output)

    full_table = pd.json_normalize(DeFi) #['tvl']['tvl'])
    #Convert data to list of dataframes
    full_table['slug'] = i
    #results = [full_table]
    frames.append(full_table)

    #Slow down requests

    time.sleep(60/50)
    #Status report
    
    print(path + i)
except ConnectionError:
    time.sleep(60)
 
single = pd.concat(frames) #Careful, this line must be outside of the for loop

print(single)

single.to_csv('DeFi_Lama.csv')
  


https://api.llama.fi/protocol/curve
https://api.llama.fi/protocol/lido
https://api.llama.fi/protocol/makerdao
    id      name                                     address symbol  \
0    3     Curve  0xD533a949740bb3306d119CC777fa900bA034cd52    CRV   
0  182      Lido  0x5a98fcbea516cf06857215779fd812ca3bef1b32    LDO   
0  118  MakerDAO  0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2    MKR   

                     url                                        description  \
0       https://curve.fi  Curve is a decentralized exchange liquidity po...   
0       https://lido.fi/  Liquidity for staked assets. Daily rewards, no...   
0  https://makerdao.com/  Builders of Dai, a digital currency that can b...   

      chain                                 logo audits audit_note  ...  \
0  Ethereum     https://icons.llama.fi/curve.png      2       None  ...   
0  Ethereum      https://icons.llama.fi/lido.png      2       None  ...   
0  Ethereum  https://icons.llama.fi/makerdao.jpg      2       N

# API Extract

In [ ]:
#@title Extract Current Data – All Protocols { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table

import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt



#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
DeFi = json.loads(output)
full_talbe = pd.json_normalize(DeFi)
full_talbe['id'] = full_talbe['symbol']  + '.' + full_talbe['name']
full_talbe['id'].replace(' ','')

full_talbe

,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,chainTvls.Polygon-offers,chainTvls.smartBCH-offers,chainTvls.Arbitrum-offers,chainTvls.zkSync,chainTvls.zkSync-offers,chainTvls.Metis-pool2,chainTvls.Harmony-Treasury,chainTvls.Metis-Treasury,chainTvls.Velas-staking,chainTvls.Celo-Treasury
0,CRV.Curve,Curve,0xD533a949740bb3306d119CC777fa900bA034cd52,CRV,https://curve.fi,Curve is a decentralized exchange liquidity po...,Multi-Chain,https://icons.llama.fi/curve.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LDO.Lido,Lido,0x5a98fcbea516cf06857215779fd812ca3bef1b32,LDO,https://lido.fi/,"Liquidity for staked assets. Daily rewards, no...",Multi-Chain,https://icons.llama.fi/lido.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MKR.MakerDAO,MakerDAO,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,MKR,https://makerdao.com/,"Builders of Dai, a digital currency that can b...",Ethereum,https://icons.llama.fi/makerdao.jpg,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ANC.Anchor,Anchor,-,ANC,https://anchorprotocol.com/,Money market on Terra,Multi-Chain,https://icons.llama.fi/anchor.jpg,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAVE.AAVE,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,https://aave.com\r\n,Aave is an Open Source and Non-Custodial proto...,Multi-Chain,https://icons.llama.fi/aave.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425,PNK.Kleros,Kleros,0x93ed3fbe21207ec2e8f2d3c3de6e058cb73bc04d,PNK,https://kleros.io,Kleros is a decentralized arbitration service ...,Multi-Chain,https://icons.llama.fi/kleros.png,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426,KDL.KDLaunch,KDLaunch,kadena:-,KDL,https://www.kdlaunch.com/#/staking,The First Decentralized IDO Platform on the Ka...,Kadena,https://icons.llama.fi/kdlaunch.png,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1427,RING.RingFi,RingFi,bsc:0x021988d2c89b1A9Ff56641b2F247942358FF05c9,RING,https://ringfi.io/,The world's first Auto-Staking & Auto-Compound...,Binance,https://icons.llama.fi/ringfi.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1428,AXL.Axl Inu,Axl Inu,0x25b24B3c47918b7962B3e49C4F468367F73CC0E0,AXL,https://www.axltoken.com,AXL INU is being developed as a Decentralized ...,Multi-Chain,https://icons.llama.fi/axl-inu.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#@title Dataset Stats { display-mode: "code" }
full_talbe.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1243 entries, 0 to 1242
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               1243 non-null   object 
 1   totalLiquidityUSD  1243 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.5+ KB


In [ ]:
#@title Historical TVL Loop (in process) { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table

import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt


path = "https://api.llama.fi/protocol/"
slug = 'aave' 
#General Information table
response = requests.get(path + slug)
output = response.text
DeFi = json.loads(output)
full_talbe = pd.json_normalize(DeFi)

tvl = full_talbe['tvl']
tvl



0    [{'date': 1578528000, 'totalLiquidityUSD': 320...
Name: tvl, dtype: object

In [ ]:
  #@title Total TVL { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table

import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt

path = "https://api.llama.fi/charts"
#slug = 'aave' 
#General Information table
response = requests.get(path)
output = response.text
DeFi = json.loads(output)
full_talbe = pd.json_normalize(DeFi)

full_talbe


,date,totalLiquidityUSD
0,1541203200,3.468404e+04
1,1541289600,4.129299e+04
2,1541376000,4.155031e+04
3,1541462400,4.323356e+04
4,1541548800,1.538626e+05
...,...,...
1238,1648166400,2.153626e+11
1239,1648252800,2.131828e+11
1240,1648339200,2.153444e+11
1241,1648425600,2.203600e+11
